In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Mecab
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter
from embedding import Embedding
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


In [71]:
data = pd.read_csv('./sample_comment.csv')
token = pd.read_csv('tokenized_0.csv')

In [72]:
df = data[['mbti','comments']]

In [73]:
for i in range(len(df)):
    if 'E' in df.loc[i, 'mbti']:
        df.loc[i, 'mbti'] = 0
    else:
        df.loc[i, 'mbti'] = 1

C:\Users\sunxo\AppData\Local\Temp\ipykernel_7756\2819924847.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i, 'mbti'] = 0
C:\Users\sunxo\AppData\Local\Temp\ipykernel_7756\2819924847.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i, 'mbti'] = 1


In [74]:
df['mbti'].value_counts() # 샘플데이터 한해서는 데이터 불균형함 

mbti
1    783
0    217
Name: count, dtype: int64

In [75]:
df = df.dropna() # 결측치 왠지모르게 있어서 제거함
token = token.dropna()

In [92]:
em = Embedding(token['comments'])
word2vec_model = em.get_embedding_model()

NameError: name 'tokenized_0' is not defined

In [99]:

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_size),
                torch.zeros(1, 1, self.hidden_size))

    def forward(self, input):
        lstm_out, self.hidden = self.lstm(input.view(len(input), 1, -1), self.hidden)
        output = self.fc(lstm_out[-1])
        return output

# 주어진 데이터셋을 바탕으로 Word2Vec 모델을 학습
embedding = Embedding(token['comments'])
word2vec_model = em.get_embedding_model()

# Word2Vec 모델의 크기를 기반으로 LSTM 모델 생성
input_size = word2vec_model.vector_size
hidden_size = 128
output_size = 1
lstm_model = LSTMModel(input_size, hidden_size, output_size)

# 학습된 Word2Vec 벡터를 가져와서 Tensor로 변환
# (여기서는 단어 벡터 중 일부를 가져올 수 있으며, 실제 데이터에서는 모든 단어 벡터를 사용하는 것이 좋습니다.)
import torch
import numpy as np

# 임의의 차원 수
vector_dimension = 100

# 단어에 대한 0 벡터 생성
def get_word_vector(word, model):
    try:
        return model.wv[word]
    except KeyError:
        return np.zeros(vector_dimension)

# 모든 단어에 대한 벡터 리스트 생성
word_vectors = [get_word_vector(word, word2vec_model) for sentence in embedding.corpus for word in sentence]

# 패딩
max_len = 86
word_vectors = [vec.tolist() + [0] * (max_len - len(vec)) for vec in word_vectors]

# 벡터 리스트를 텐서로 변환
input_tensor = torch.tensor(word_vectors, dtype=torch.float32)


# LSTM 모델에 입력 전달하여 출력 받기
output = lstm_model(input_tensor)
print("LSTM 모델의 출력:", output)

[I 2023-12-09 22:40:27,817] A new study created in memory with name: no-name-3126e368-fb65-4c71-acbc-995f9d8957a1
[I 2023-12-09 22:40:27,865] Trial 0 finished with value: 33503.30859375 and parameters: {'vector_size': 91, 'window': 3, 'min_count': 26, 'sg': 0}. Best is trial 0 with value: 33503.30859375.


ValueError: expected sequence of length 100 at dim 1 (got 91)

In [89]:
'감정' in word2vec_model.wv

True

In [93]:
embedding.corpus

[['눈빛',
  '과',
  '기류',
  '를',
  '보면',
  '알',
  '지',
  '않나용',
  '일단',
  '진심',
  '으로',
  '이뻐하는',
  '게',
  '여기',
  '까지',
  '느껴지네요'],
 ['저', '도', '눈빛', '과', '기류', '를', '보고', '알수있다고', '생각', '해요'],
 ['걱정',
  '하지',
  '마세요',
  '엔',
  '프제',
  '라고',
  '해서',
  '다',
  '조리',
  '있게',
  '말',
  '하',
  '지는',
  '않더라구요',
  '대신',
  '공통점',
  '이',
  '칭찬',
  '은',
  '잘',
  '하더라구요'],
 ['네', '말', '도', '많구', '말', '에', '칭찬', '이', '많이', '차지', '하더라구요'],
 ['밀당', '아닌가요'],
 ['저',
  '분들',
  '심리',
  '를',
  '공부',
  '를',
  '좀',
  '해보니',
  '밀당',
  '은',
  '또',
  '아닌거',
  '같더라구요',
  '저희',
  '랑',
  '생각',
  '자체',
  '가',
  '너무',
  '달랐어요',
  '멀어지려고',
  '하면',
  '쓱',
  '다가오고',
  '그래서',
  '다가가면',
  '또',
  '멀어지고',
  '뭔가',
  '싶네요'],
 ['애정결핍', '같기도'],
 ['제', '글', '한번', '반', '보고', '와주시면', '감사하겠습니다'],
 ['스며들', '기하', '는', '법좀', '알려', '즈', '세', '요'],
 ['와', '멋있어요', '직접', '그', '리신', '건가', '요', '감탄', '하고', '갑니다'],
 ['멋져요'],
 ['대박'],
 ['오',
  '맞아요',
  '인',
  '프제',
  '가',
  '친절하기만',
  '하면',
  '별로',
  '안',
  '친한거고',
  '저런',
  '장난',
  '치면',
  '